In [ ]:
### Download, Inspect and Upload Permit Data
from open_mastr import Mastr
import pandas as pd
import geopandas as gpd
import supabase_py
import psycopg2
import sqlalchemy
import os
import numpy as np
from dotenv import load_dotenv
from pandas.api.types import is_datetime64_any_dtype
from datetime import datetime
import time

In [ ]:
### Data was already downloaded with open_mastr into local sqllite DB
db = Mastr()
conn = db.engine # Connection engine

df_permit = pd.read_sql_table("permit", conn)
columns_permit = list(df_permit.columns)

In [ ]:
### function to turn CamelCase to snake_case
def change_case(str):
    # List comprehension, starts with an _ wich is removed by lstrip("_")
    # loops through word, if upper, _ first "_"+
    # and i.lower() as a string method
    # just return i else
    return ''.join(['_'+i.lower() if i.isupper() 
               else i for i in str]).lstrip('_')
    
def dtype_sqltype(str, map_dict):
    
    # next() jumps through the iterator until a match is found
    # with a an iterator generated by the comprehension inside ()
    return next((key for key, val in map_dict.items() if val == str), None)

In [ ]:
### How many are linked to windpower-data?
df_permit.info() # VerknuepfteEinheiten as Foreign-Key column

### Open Windpower-Data
### Connect to the database
# downloaded certiticate
# Set connection details in .env

# Load environment variables from .env file
load_dotenv()

# Get connection parameters from environment variables
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
ssl_cert_path = os.getenv("SSL_CERT_PATH")

# Construct the connection string
conn_str = f"dbname={dbname} user={user} password={password} host={host} port={port} sslmode=require sslrootcert={ssl_cert_path}"

# Etablish connection object

#conn.close()
conn = psycopg2.connect(conn_str)

In [ ]:
### Get windpower-Data
# Mastr-Einheiten-Nummer
# Spalte in permit verknüpfte Einheiten als array
# Welche enthalten Wind-Einheiten -> Sind unter diesen jeweils alle eines Arrays bekannte Windeinheiten?

# Nur die mit Wind in die Datenbank schreiben, 
# Genehmigungsnummer als FK zu permits als Spalte in wind_extended hinzufügen
# Welche Constraints sind sinnvoll?

In [ ]:
sql_select = "SELECT * FROM public.wind_extended;"
df_wind = pd.read_sql(sql_select, conn)

In [ ]:
df_permit.info()

In [ ]:
# One item of df_permit["Verknuefte_Einheiten"]: "hdjsahd98698, 875875hjhjhj, 65654dhdf"
rows = []

for index,row in df_permit.iterrows():
    permit = row["GenMastrNummer"]
    units = row["VerknuepfteEinheiten"].split(", ")
    
    for unit in units:
        rows.append({"GenMastrNummer": permit, "Einheit": unit})

df_permits_units = pd.DataFrame(rows)
df_permits_units.info()

In [27]:
cols_keep = ['id', 'name_windpark','eeg_mastr_nummer',
             'einheit_systemstatus',
       'einheit_betriebsstatus',      
       'einheit_mastr_nummer', 
       'registrierungsdatum',
       'meldedatum', 'geplantes_inbetriebnahmedatum', 'inbetriebnahmedatum',
       'datum_endgueltige_stilllegung',
       'datum_beginn_voruebergehende_stilllegung',
       'datum_beendigung_vorlaeufigen_stilllegung',
       'datum_wiederaufnahme_betrieb',
       'datum_letzte_aktualisierung']
df_wind_perm = df_wind[cols_keep]

In [30]:
df_wind_perm.columns

Index(['id', 'name_windpark', 'eeg_mastr_nummer', 'einheit_systemstatus',
       'einheit_betriebsstatus', 'einheit_mastr_nummer', 'registrierungsdatum',
       'meldedatum', 'geplantes_inbetriebnahmedatum', 'inbetriebnahmedatum',
       'datum_endgueltige_stilllegung',
       'datum_beginn_voruebergehende_stilllegung',
       'datum_beendigung_vorlaeufigen_stilllegung',
       'datum_wiederaufnahme_betrieb', 'datum_letzte_aktualisierung'],
      dtype='object')

In [40]:
df_permits_wind = pd.merge(df_permits_units, df_wind_perm, 
                           left_on="Einheit", right_on="einheit_mastr_nummer",
                           how="outer")

In [37]:
df_permits_wind

,GenMastrNummer,Einheit,id,name_windpark,eeg_mastr_nummer,einheit_systemstatus,einheit_betriebsstatus,einheit_mastr_nummer,registrierungsdatum,meldedatum,geplantes_inbetriebnahmedatum,inbetriebnahmedatum,datum_endgueltige_stilllegung,datum_beginn_voruebergehende_stilllegung,datum_beendigung_vorlaeufigen_stilllegung,datum_wiederaufnahme_betrieb,datum_letzte_aktualisierung
0,SGE961876398816,SEE940146675093,11.0,Windpark Kreuzstein,EEG951718125489,Aktiviert,In Betrieb,SEE940146675093,2019-02-13,None,None,2017-09-01,None,None,None,None,2022-11-17
1,SGE961876398816,SEE914108319653,13.0,Windpark Kreuzstein,EEG996216581153,Aktiviert,In Betrieb,SEE914108319653,2019-02-13,None,None,2017-09-04,None,None,None,None,2022-11-17
2,SGE961876398816,SEE982417853618,14.0,Windpark Kreuzstein,EEG926639129328,Aktiviert,In Betrieb,SEE982417853618,2019-02-13,None,None,2017-08-31,None,None,None,None,2022-11-17
3,SGE961876398816,SEE983797054244,142.0,Windpark Kreuzstein,EEG956425865284,Aktiviert,In Betrieb,SEE983797054244,2019-02-13,None,None,2017-05-31,None,None,None,None,2022-11-17
4,SGE961876398816,SEE916761594682,145.0,Windpark Kreuzstein,EEG968558404292,Aktiviert,In Betrieb,SEE916761594682,2019-02-13,None,None,2017-06-26,None,None,None,None,2022-11-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56861,NaN,NaN,35226.0,Osterholm,None,Aktiviert,In Planung,SEE931208588833,2023-11-08,None,2023-12-03,None,None,None,None,None,2023-11-08
56862,NaN,NaN,35235.0,WEA Berk,None,Aktiviert,In Planung,SEE985840782487,2023-11-08,None,2025-09-30,None,None,None,None,None,2023-11-08
56863,NaN,NaN,35236.0,WEA Berk,None,Aktiviert,In Planung,SEE939753516752,2023-11-08,None,2025-09-30,None,None,None,None,None,2023-11-08
56864,NaN,NaN,35237.0,WEA Berk,None,Aktiviert,In Planung,SEE972667490619,2023-11-08,None,2025-09-30,None,None,None,None,None,2023-11-08


In [ ]:
# Wie viele aus Wind haben eine Genehmigung
# Wie ist der Betriebsstatus

# Wie viele aus Permit sind in Wind?
# Gibt es Dopplungen:
# Eine Anlage aus Wind -> mehrere Genehmigungen?
# Eine Genehmigung